In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings

In [3]:
plt.rcParams['font.size'] = 14
warnings.simplefilter('ignore')

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [14]:
# シンボリックリンクによって別階層のファイルをimportする
from pythonlibs.torch_lib1 import *
print(README)

Common Library for PyTorch
Author: M. Akaishi


In [22]:
!tree pythonlibs/images/dog_wolf

pythonlibs/images/dog_wolf
├── test
│   ├── dog
│   │   ├── dog-21.png
│   │   ├── dog-22.png
│   │   ├── dog-23.png
│   │   ├── dog-24.png
│   │   └── dog-25.png
│   └── wolf
│       ├── wolf-21.png
│       ├── wolf-22.png
│       ├── wolf-23.png
│       ├── wolf-24.png
│       └── wolf-25.png
└── train
    ├── dog
    │   ├── dog-01.png
    │   ├── dog-02.png
    │   ├── dog-03.png
    │   ├── dog-04.png
    │   ├── dog-05.png
    │   ├── dog-06.png
    │   ├── dog-07.png
    │   ├── dog-08.png
    │   ├── dog-09.png
    │   ├── dog-10.png
    │   ├── dog-11.png
    │   ├── dog-12.png
    │   ├── dog-13.png
    │   ├── dog-14.png
    │   ├── dog-15.png
    │   ├── dog-16.png
    │   ├── dog-17.png
    │   ├── dog-18.png
    │   ├── dog-19.png
    │   └── dog-20.png
    └── wolf
        ├── wolf-01.png
        ├── wolf-02.png
        ├── wolf-03.png
        ├── wolf-04.png
        ├── wolf-05.png
        ├── wolf-06.png
        ├── wolf-07.png
        ├── wolf-08.png
        ├── wolf-

In [ ]:
# 事前学習済みモデルを利用したモデル構築
from torchvision import models
net = models.vgg19_bn(pretrained=True)

# 転移学習のためのrequires_gradをFalseに設定
for param in net.parameters():
    param.requires_grad = False

# 乱数初期化
torch_seed()

# 最終ノードの出力を2に変更
in_features = net.classifier[6].in_features
net.classifier[6] = nn.Linear(in_features, 2)

# AdaptiveAvgPool2d関数の取り外し
net.avgpool = nn.Identity()

# GPU利用
net = net.to(device)

# 学習率
lr = 0.001

# 損失関数
criterion = nn.CrossEntropyLoss()

# 最適化関数の定義
# パラメータ修正の対象を最終ノードに限定
optimizer = optim.SGD(net.classifier[6].parameters(), lr=lr, momentum=0.9)

# historyファイルの初期化
history = np.zeros((0, 5))

# 繰り返し回数
num_epochs = 10

# 学習
history = fit(net, optimizer, criterion, num_epochs, train_loader, test_loader, device, history)